In [11]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [25]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
import torch

class TrainDataset(Dataset): 
    
    def __init__(self):
        with open('./Network dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader) #속성 85개
        col1 = 4
        col2 = 7
        target_list = [ 0 if data[i][84] == 'BENIGN' else 1 for i in range(len(data))]
        target_arr = np.array(target_list[1000:-1000])
        data_arr = np.array(data[1000:-1000])
        attribute = np.array(data[0]) 
        #Benign 0, DDos 1
        self.len = len(target_arr) 
        self.X = torch.tensor(np.array(data_arr[:,[col1,col2]]).astype(np.float32)) # 시각화를 위해 속성 2개만 선정
        self.y = torch.tensor(target_arr, dtype=torch.float32) 
    
    def __getitem__(self, index): 
        
        return self.X[index], self.y[index] 
    
    def __len__(self): 
        
        return self.len
    
class TestDataset(Dataset): 
    
    def __init__(self):
        with open('./Network dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
        col1 = 4
        col2 = 7
        target_list = [ 0 if data[i][84] == 'BENIGN' else 1 for i in range(len(data))]
        target_arr = np.array(target_list[1:1000] + target_list[-1000:])
        data_arr = np.array(data[1:1000] + data[-1000:])
        attribute = np.array(data[0]) 
        #Benign 0, DDos 1
        self.len = len(target_arr) 
        self.X = torch.tensor(np.array(data_arr[:,[col1,col2]]).astype(np.float32)) # 시각화를 위해 속성 2개만 선정
        self.y = torch.tensor(target_arr, dtype=torch.float32) 
    
    def __getitem__(self, index): 
        
        return self.X[index], self.y[index] 
    
    def __len__(self): 
        
        return self.len

trainset = TrainDataset()
testset = TestDataset()


trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('Benign', 'DDos')

In [29]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.dropout = torch.nn.Dropout()
        
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        #x = nn.Sigmoid(x)
        return x


net = Net()

In [30]:
import torch.optim as optim

criterion = nn.BCELoss()
#SGD 옵티마이져 별로임
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += float(loss.item())
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[{}, {}] loss: {}'.format(epoch + 1, i + 1, running_loss/2000))
            #너무 작아서 2000나누지 않고 보기
            running_loss = 0.0

print('Finished Training')

[1, 2000] loss: 15.786832258976997
[1, 4000] loss: 16.04205815458298
[1, 6000] loss: 15.639336456775665
[1, 8000] loss: 16.02965051364899
[1, 10000] loss: 15.683766849279404
[1, 12000] loss: 15.706601897716522
[1, 14000] loss: 15.779896235704422
[1, 16000] loss: 15.949788195252419
[1, 18000] loss: 16.014518974661826
[1, 20000] loss: 15.687988189935684
[1, 22000] loss: 15.784739001750946
[1, 24000] loss: 15.654054911136628
[1, 26000] loss: 15.984825105682015
[1, 28000] loss: 15.70744383263588
[1, 30000] loss: 15.79112894153595
[1, 32000] loss: 15.925830174207688
[1, 34000] loss: 15.90856078338623
[1, 36000] loss: 15.55626525402069
[1, 38000] loss: 15.704782025575637
[1, 40000] loss: 15.88730639386177
[1, 42000] loss: 15.982246567249298
[1, 44000] loss: 16.17868946146965
[1, 46000] loss: 15.876715893268585
[1, 48000] loss: 15.801490576982498


In [9]:
incorrect = 0
total = 0
print(len(testset))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        outputs[outputs > 0.5] = 1
        outputs[outputs <= 0.5] = 0
        result = abs(labels - outputs)
        total += len(outputs)
        incorrect += result.sum()
        
print('Accuracy of the network on the 1999 test data: %d %%' % (
    100 * (total - incorrect) / total))

1999
Accuracy of the network on the 1999 test data: 100 %


In [18]:
target_list = [ 0 if data[i][84] == 'BENIGN' else 1 for i in range(len(data))]

In [22]:
data[1][84]
data[0][84]

' Label'

In [23]:
print(target_list[10000:20000])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 